# Tests MSSQL Python

Frist create a .env file:

```env
MSSQL_SERVER="..."
MSSQL_DATABASE=""..."
MSSQL_UID="..."
MSSQL_PWD="..."
```

In [4]:
%reload_ext dotenv
%dotenv

import os

config = {
    "server": os.environ["MSSQL_SERVER"],
    "database": os.environ["MSSQL_DATABASE"],
    "uid": os.environ["MSSQL_UID"],
    "pwd": os.environ["MSSQL_PWD"],
}

## PyODBC

**Microsoft ODBC Driver for SQL Server is required before to install pyodbc**.

Please follow the instructions to install the driver for your OS. See the [docs](https://learn.microsoft.com/en-us/sql/connect/odbc/microsoft-odbc-driver-for-sql-server) for details.

To install run this command:
```bash
poetry add pyodbc
```

In [5]:
import pyodbc

sql_server_drivers = list(filter(lambda x: "SQL Server" in x, pyodbc.drivers()))
conn_str = (
    f"DRIVER={{{sql_server_drivers[0]}}};"
    f'SERVER=tcp:{config["server"]};PORT=1433;'
    f'DATABASE={config["database"]};'
    f'UID={config["uid"]};'
    f'PWD={config["pwd"]}'
)
display(conn_str)

with pyodbc.connect(conn_str) as conn:
    with conn.cursor() as cursor:
        cursor.execute("select 1")

'DRIVER={ODBC Driver 17 for SQL Server};SERVER=tcp:ap50-tst-datal-sqlmi01.33b9b5988eba.database.windows.net;PORT=1433;DATABASE=STMS_DataLake_VN;UID=datafactory;PWD=7TE7rxMnTwM9V'

OperationalError: ('HYT00', '[HYT00] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0) (SQLDriverConnect)')

## PyTDS

To install run this command:

```bash
poetry add python-tds
```

If you want to use TLS you should also install pyOpenSSL package:

```bash
poetry add pyOpenSSL
```

For a better performance install bitarray package too:

```bash
poetry add bitarray
```

To use Kerberos on non-Windows platforms (experimental) install kerberos package:

```bash
poetry add kerberos
```

In [4]:
import pytds

with pytds.connect(dsn=config["server"], database=config["database"], user=config["uid"], password=config["pwd"]) as conn:
    with conn.cursor() as cursor:
        cursor.execute("select 1")

Work attempt exceeded it's allocated time 1.200000, actual time was 1.236883.


LoginError: ("Cannot connect to server 'ap50-tst-datal-sqlmi01.33b9b5988eba.database.windows.net': timed out", None)

## MiniAB

**miniab** requires duckdb and pyarrow and they should be installed before to use it.

To install duckdb and pyarrow run this command:

```bash
poetry add duckdb pyarrow
```

In [3]:
# import duckdb
import miniab as ab

source = ab.get_source(
    "source-mssql",
    config = config,
    sync = True
)
source.select_all_streams()
source.check()

cache = ab.get_default_cache()
result = source.read(cache=cache)

# dim_pack_tbl = result.to_arrow("dim_pack")
# duckdb.sql("SELECT * FROM dim_pack_tbl LIMIT 10;").show()

Work attempt exceeded it's allocated time 1.200000, actual time was 1.236858.


LoginError: ("Cannot connect to server 'ap50-tst-datal-sqlmi01.33b9b5988eba.database.windows.net': timed out", None)